# Homework 4
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories.

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work.

In [1]:
import platform
display(platform.system())
import os
file_download_link = 'https://www.dropbox.com/scl/fi/j1dxtqjerbdbl81e05nvp/hw4data.zip?rlkey=8mkxz4j8lngziok6on782a8u4&dl=0'
if os.name == 'nt':
    print('Please download your dataset here:', file_download_link)
else:
    # We need to first download the data here:
    !wget -O data.zip "$file_download_link" -o /dev/null
    !unzip data.zip > /dev/null

'Windows'

Please download your dataset here: https://www.dropbox.com/scl/fi/j1dxtqjerbdbl81e05nvp/hw4data.zip?rlkey=8mkxz4j8lngziok6on782a8u4&dl=0


In [2]:
!sed 's/,/\t/g' garments_worker_productivity.csv > garments_worker_productivity.tsv

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
# If your data is on google drive then uncomment the code below to access
# your google drive.
#from google.colab import drive
#drive.mount('/content/drive')
#!pip install cvxpy
#!pip install -U imbalanced-learn

In [4]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install imbalanced-learn delayed

# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`.

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - Fill the empty values with 0.


In [5]:
import pandas as pd
# If put the data(.csv) under the same folder, you could use
# df = pd.read_csv('./garments_worker_productivity.csv')
df = pd.read_csv('./garments_worker_productivity.csv')
df.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [6]:
import sklearn as sk
import numpy as np
import scipy as sp
import cvxpy as cp
from sklearn.model_selection import train_test_split
import imblearn

categorical_cols = ['day','quarter','department','team']


#Drop the column date.
df = df.drop('date', axis=1) 


#For each of the categorical attributes, print out all the unique elements.
print('before remap:')
for col in categorical_cols:
    print(col, df[col].unique())
    
    
#For each of the categorical attributes, remap the duplicated items,
#if you find there are typos or spaces among the duplicated items.
df['department'] = df['department'].replace({'sweing':'sewing', 'finishing ':'finishing'})
print('\n after remap:')
for col in ['day','quarter','department','team']:
    print(col, df[col].unique())
    
    
#Create another column named satisfied that records the productivity performance. 
df['satisfied'] = df.apply(lambda x: 1 if x['actual_productivity'] >= x['targeted_productivity'] else 0, axis=1)
print('\n satisfied unique vals',df.satisfied.unique())


#Drop the columns actual_productivity and targeted_productivity.
df = df.drop(['actual_productivity', 'targeted_productivity'], axis=1)


#Find and print out which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
print('\n cols with empty values: \n', df.columns[df.isna().any()].tolist())


#Fill the empty values with 0.
df_p1 = df.fillna(0)
df_p1.head(10)

before remap:
day ['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
quarter ['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']
department ['sweing' 'finishing ' 'finishing']
team [ 8  1 11 12  6  7  2  3  9 10  5  4]

 after remap:
day ['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
quarter ['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']
department ['sewing' 'finishing']
team [ 8  1 11 12  6  7  2  3  9 10  5  4]

 satisfied unique vals [1 0]

 cols with empty values: 
 ['wip']


,quarter,department,day,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,Quarter1,sewing,Thursday,8,26.16,1108.0,7080,98,0.0,0,0,59.0,1
1,Quarter1,finishing,Thursday,1,3.94,0.0,960,0,0.0,0,0,8.0,1
2,Quarter1,sewing,Thursday,11,11.41,968.0,3660,50,0.0,0,0,30.5,1
3,Quarter1,sewing,Thursday,12,11.41,968.0,3660,50,0.0,0,0,30.5,1
4,Quarter1,sewing,Thursday,6,25.90,1170.0,1920,50,0.0,0,0,56.0,1
5,Quarter1,sewing,Thursday,7,25.90,984.0,6720,38,0.0,0,0,56.0,1
6,Quarter1,finishing,Thursday,2,3.94,0.0,960,0,0.0,0,0,8.0,1
7,Quarter1,sewing,Thursday,3,28.08,795.0,6900,45,0.0,0,0,57.5,1
8,Quarter1,sewing,Thursday,2,19.87,733.0,6000,34,0.0,0,0,55.0,1
9,Quarter1,sewing,Thursday,1,28.08,681.0,6900,45,0.0,0,0,57.5,1


## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attribtues**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.

In [7]:
# Remember to continue the task with your processed data from Exercise 1

df = df_p1.copy()
#For each of the categorical attribtues, encode the set of categories to be 0 ~ (n_classes - 1)
le = sk.preprocessing.LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])


X = df.drop('satisfied',axis=1)
y = df['satisfied']

#Split the data into training and testing set with the ratio of 80:20.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=12)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

df.head()

(957, 12) (240, 12) (957,) (240,)


,quarter,department,day,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,0,1,3,7,26.16,1108.0,7080,98,0.0,0,0,59.0,1
1,0,0,3,0,3.94,0.0,960,0,0.0,0,0,8.0,1
2,0,1,3,10,11.41,968.0,3660,50,0.0,0,0,30.5,1
3,0,1,3,11,11.41,968.0,3660,50,0.0,0,0,30.5,1
4,0,1,3,5,25.90,1170.0,1920,50,0.0,0,0,56.0,1


### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [8]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report


X_cat_train = X_train[categorical_cols] #categorical X train
X_cat_test = X_test[categorical_cols] #categorical X test
cat_nb = CategoricalNB()
cat_nb.fit(X_cat_train, y_train)
y_cat_pred = cat_nb.predict(X_cat_test)

#Report the testing result using classification_report
print(classification_report(y_test, y_cat_pred))

              precision    recall  f1-score   support

           0       0.64      0.23      0.33        62
           1       0.78      0.96      0.86       178

    accuracy                           0.77       240
   macro avg       0.71      0.59      0.60       240
weighted avg       0.74      0.77      0.72       240



### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [9]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer

scaler = QuantileTransformer(output_distribution='normal')
gnb = GaussianNB()

X_num_train = X_train.drop(categorical_cols, axis=1)
X_num_test = X_test.drop(categorical_cols, axis=1)

scaler = scaler.fit(X.drop(categorical_cols, axis=1)) #fitting to whole dataset
X_num_train_scaled = scaler.transform(X_num_train)
X_num_test_scaled = scaler.transform(X_num_test)
gnb.fit(X_num_train_scaled,y_train)

y_num_pred = gnb.predict(X_num_test_scaled)

#Report the testing result using classification_report
print(classification_report(y_test, y_num_pred))

              precision    recall  f1-score   support

           0       0.77      0.32      0.45        62
           1       0.80      0.97      0.88       178

    accuracy                           0.80       240
   macro avg       0.79      0.64      0.67       240
weighted avg       0.79      0.80      0.77       240



## Exercies 3 - SVM Classifier (35 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attribtues**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.


In [10]:
# Remember to continue the task with your processed data from Exercise 1

df = df_p1.copy()
X = df.drop('satisfied',axis=1)
y = df['satisfied']

#For each of the categorical attribtues, encode them with one-hot encoding.
X_ohe = pd.get_dummies(X, columns=categorical_cols,dtype=float)
print(X_ohe.columns)

#Split the data into training and testing set with the ratio of 80:20.
X_train, X_test, y_train, y_test = train_test_split(X_ohe,y,test_size=0.2,random_state=12)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train.head()


Index(['smv', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men',
       'no_of_style_change', 'no_of_workers', 'day_Monday', 'day_Saturday',
       'day_Sunday', 'day_Thursday', 'day_Tuesday', 'day_Wednesday',
       'quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3',
       'quarter_Quarter4', 'quarter_Quarter5', 'department_finishing',
       'department_sewing', 'team_1', 'team_2', 'team_3', 'team_4', 'team_5',
       'team_6', 'team_7', 'team_8', 'team_9', 'team_10', 'team_11',
       'team_12'],
      dtype='object')
(957, 33) (240, 33) (957,) (240,)


,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,day_Monday,day_Saturday,...,team_3,team_4,team_5,team_6,team_7,team_8,team_9,team_10,team_11,team_12
1183,21.82,1674.0,6000,34,0.0,0,0,50.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
891,30.10,709.0,6960,45,0.0,0,1,58.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
161,11.41,887.0,5490,50,0.0,0,0,30.5,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
959,2.90,0.0,3360,0,0.0,0,0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
662,3.94,0.0,2160,0,0.0,0,0,18.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Exercise 3.2 - SVM with Different Kernels (15 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [11]:
# Remember to do this task with your processed data from Exercise 3.1

from sklearn.svm import SVC

#Build one SVM with linear kernel.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Build one SVM with linear kernel.
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train_scaled, y_train)
pred_linear = svm_linear.predict(X_test_scaled)
print('SVM with linear kernel: \n', classification_report(y_test, pred_linear))

#Build another SVM but with rbf kernel.
svm_rbf = SVC(kernel='rbf')
svm_rbf.fit(X_train_scaled, y_train)
pred_rbf = svm_rbf.predict(X_test_scaled)
print('SVM with rbf kernel: \n', classification_report(y_test,pred_rbf))

SVM with linear kernel: 
               precision    recall  f1-score   support

           0       1.00      0.03      0.06        62
           1       0.75      1.00      0.86       178

    accuracy                           0.75       240
   macro avg       0.87      0.52      0.46       240
weighted avg       0.81      0.75      0.65       240

SVM with rbf kernel: 
               precision    recall  f1-score   support

           0       0.68      0.34      0.45        62
           1       0.80      0.94      0.87       178

    accuracy                           0.79       240
   macro avg       0.74      0.64      0.66       240
weighted avg       0.77      0.79      0.76       240



### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class.
 - Oversample the **training data**.
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br >
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [12]:
# Remember to do this task with your processed data from Exercise 3.1
from imblearn.over_sampling import RandomOverSampler, SMOTE,ADASYN

#For the column satisfied in our training set, please print out the frequency of each class.
print('training set frequency not oversampled:')
print(y_train.value_counts())

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Oversample the training data.
ovs = RandomOverSampler()
X_train_over, y_train_over = ovs.fit_resample(X_train_scaled,y_train)

#For the column satisfied in the oversampled data, print out the frequency of each class again.
print('\ntraining set frequency oversampled')
print(y_train_over.value_counts())

training set frequency not oversampled:
satisfied
1    697
0    260
Name: count, dtype: int64

training set frequency oversampled
satisfied
1    697
0    697
Name: count, dtype: int64


In [13]:
#Re-build the same 2 SVMs using oversampled training data instead.

#linear
svm_linear_new = SVC(kernel='linear')
svm_linear_new.fit(X_train_over, y_train_over)
pred_linear_new = svm_linear_new.predict(X_test_scaled)
print('\nSVM with linear kernel: \n', classification_report(y_test, pred_linear_new))

#rbf
svm_rbf_new = SVC(kernel='rbf',C=3)
svm_rbf_new.fit(X_train_over, y_train_over)
pred_rbf_new = svm_rbf_new.predict(X_test_scaled)
print('SVM with rbf kernel: \n', classification_report(y_test,pred_rbf_new))


SVM with linear kernel: 
               precision    recall  f1-score   support

           0       0.44      0.76      0.56        62
           1       0.89      0.66      0.76       178

    accuracy                           0.69       240
   macro avg       0.66      0.71      0.66       240
weighted avg       0.77      0.69      0.71       240

SVM with rbf kernel: 
               precision    recall  f1-score   support

           0       0.44      0.56      0.49        62
           1       0.83      0.75      0.79       178

    accuracy                           0.70       240
   macro avg       0.63      0.66      0.64       240
weighted avg       0.73      0.70      0.71       240



## 4) Collaborative Statement (5 points)
#### You must fill this out even if you worked alone to get credit.

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

I didn't talk to anyone!